# TNFR Phase-Gated Coupling Demo (U3) — Telemetry-only

This cellbook mirrors `examples/phase_gated_coupling_demo.py`. It shows explicit phase verification (U3) gating for coupling between two atom-like graphs.

Scenarios:
1) In-phase terminals (compatible) → edge added
2) Anti-phase terminals (incompatible) → edge blocked

We compute Structural Field Tetrad telemetry and a sequential ΔΦ_s check (U6, read-only), then render quick plots coloring nodes by atom and annotating gating.

In [ ]:
from __future__ import annotations

import os, math, json
from typing import Any, Dict, Tuple

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

from tnfr.examples_utils import build_element_radial_graph, apply_synthetic_activation_sequence
from tnfr.metrics.phase_compatibility import is_phase_compatible, compute_phase_coupling_strength
from tnfr.physics.fields import (
    compute_structural_potential, estimate_coherence_length,
)
from tnfr.operators.grammar import (
    warn_phase_gradient_telemetry, warn_phase_curvature_telemetry, warn_coherence_length_telemetry,
    validate_structural_potential_confinement,
)
from tnfr.telemetry.constants import (
    PHASE_GRADIENT_THRESHOLD, PHASE_CURVATURE_ABS_THRESHOLD, STRUCTURAL_POTENTIAL_DELTA_THRESHOLD,
)

OUTPUT_DIR = os.path.join('examples','output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

def _pick_terminal(G: nx.Graph) -> int:
    cands = [n for n, d in G.nodes(data=True) if d.get('role') == 'shell1']
    if not cands:
        cands = [n for n, d in G.nodes(data=True) if d.get('role') != 'nucleus']
    return sorted(cands)[0]

def _telemetry_row(G: nx.Graph, label: str) -> Dict[str, Any]:
    phi_before = compute_structural_potential(G)
    _, stats_g, msg_g, _ = warn_phase_gradient_telemetry(G, threshold=PHASE_GRADIENT_THRESHOLD)
    _, stats_k, msg_k, _ = warn_phase_curvature_telemetry(G, abs_threshold=PHASE_CURVATURE_ABS_THRESHOLD, multiscale_check=True, alpha_hint=2.76)
    _, stats_x, msg_x = warn_coherence_length_telemetry(G)
    xi_c = float(estimate_coherence_length(G))

    apply_synthetic_activation_sequence(G, alpha=0.25, dnfr_factor=0.9)
    phi_after = compute_structural_potential(G)
    ok, drift, msg_u6 = validate_structural_potential_confinement(
        G, phi_before, phi_after, threshold=STRUCTURAL_POTENTIAL_DELTA_THRESHOLD, strict=False
    )
    return {
        'label': label,
        'xi_c': xi_c,
        'mean_grad': float(stats_g.get('mean_abs', 0.0)),
        'mean_kphi': float(stats_k.get('mean_abs', 0.0)),
        'mean_path_length': float(stats_x.get('mean_path_length', 0.0)),
        'u6_ok': bool(ok),
        'u6_drift': float(drift),
        'telemetry_msgs': [msg_g, msg_k, msg_x, msg_u6],
    }

def _color_map(G: nx.Graph):
    colors = {'A': '#1f77b4', 'B': '#ff7f0e'}
    return [colors.get(G.nodes[n].get('atom','?'), '#7f7f7f') for n in G.nodes()]

def _plot_graph(G: nx.Graph, title: str):
    pos = nx.spring_layout(G, seed=123)
    node_colors = _color_map(G)
    plt.figure(figsize=(6, 4))
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=60)
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# Build two element-like graphs (H and O), merge, and tag atoms for plotting
GA = build_element_radial_graph(1, seed=123)  # H-like
GB = build_element_radial_graph(8, seed=124)  # O-like
offB = max(GA.nodes()) + 1
GB_rel = nx.relabel_nodes(GB, {n: n + offB for n in GB.nodes()})
G = nx.Graph(); G.update(GA); G.update(GB_rel)
for n in GA.nodes(): G.nodes[n]['atom'] = 'A'
for n in GB_rel.nodes(): G.nodes[n]['atom'] = 'B'
a = _pick_terminal(GA)
b = _pick_terminal(GB_rel)
a, b

In [ ]:
rows = []

# Scenario 1: in-phase (compatible)
G.nodes[a]['phase'] = 0.0
G.nodes[b]['phase'] = 0.0
cs = compute_phase_coupling_strength(G.nodes[a]['phase'], G.nodes[b]['phase'])
compat = is_phase_compatible(G.nodes[a]['phase'], G.nodes[b]['phase'], threshold=0.9)
if compat: G.add_edge(a, b, gated_by='U3', coupling_strength=cs)
rows.append(_telemetry_row(G, f'in-phase (cs={cs:.2f}, compat={compat})'))
_plot_graph(G, title=f'In-phase: edge_added={G.has_edge(a,b)}')

# Reset edge for scenario 2
if G.has_edge(a,b): G.remove_edge(a,b)

# Scenario 2: anti-phase (incompatible)
G.nodes[a]['phase'] = 0.0
G.nodes[b]['phase'] = math.pi
cs = compute_phase_coupling_strength(G.nodes[a]['phase'], G.nodes[b]['phase'])
compat = is_phase_compatible(G.nodes[a]['phase'], G.nodes[b]['phase'], threshold=0.9)
if compat: G.add_edge(a, b, gated_by='U3', coupling_strength=cs)
rows.append(_telemetry_row(G, f'anti-phase (cs={cs:.2f}, compat={compat})'))
_plot_graph(G, title=f'Anti-phase: edge_added={G.has_edge(a,b)}')

pd.DataFrame(rows)[['label','xi_c','mean_grad','mean_kphi','mean_path_length','u6_drift','u6_ok']]

In [ ]:
# Save JSONL (parity with script)
jsonl_path = os.path.join(OUTPUT_DIR, 'phase_gated_coupling_demo.jsonl')
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")
jsonl_path